###### COMSOL Study: Mesh refinement for current vacuum chamber
## Electric field strength along x axis
- current chamber
- COMSOL files 01.MR - current vacuum chamber with microwave ion source (1 mm aperture)
- Mesh is mesh surface contraints
- more refined mesh

## Import statements

In [30]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import sys
from scipy.interpolate import interp1d

### Specify path to datafile


In [31]:
remote_path = '/Users/hkromer/02_PhD/02_Data/01_COMSOL/01_IonOptics/\
03.new_chamber/01.MR/more_refined_mesh/lineData/'
files_along_x = f'{remote_path}'

### Parameter Settings

In [32]:
ylimit = 7.5  # ylimit for the plot
X_max = 80 # maximum x point to interpolate the data (target surface)
hmax_reference = 0.8 # reference hmax which will be used as the reference for the electric field strength (relative differences will be plotted)  check in NOTION

### Load the files

In [33]:
files = os.listdir(files_along_x)
# files = [f for f in files if f.endswith('.csv')]
files = [f for f in files if f.endswith('.csv')]
# print(files)
# sys.exit()

# process files
# (1): load x value and value of electric field (es.normE) as well as potential field (V)
df = pd.DataFrame()
for file in files:
	print('Processing file: {}'.format(file))
	if len(df) < 1:
		df = pd.read_csv('{}/{}'.format(files_along_x, file),
			header=None, skiprows=9)
		# print(df)
		colname = ['x', 'y', 'z', 'ElField']  # mm, mm, mm, mm, kV/mm
		df.columns = colname
		fname = re.findall(r'(.+).csv', file)[0]
		run = re.findall(r'\.h(\d\.\d)\.', file)[0]
		# print(run)
		# hmax = re.findall(r'hmax(\d*)', file)[0]
		hmax = run
		df['ID'] = fname
		df['run'] = run
		df['hmax'] = int(hmax[-1])
		df = df.sort_values(by=['x'])
		df = df.reset_index()
	else:
		this_df = pd.read_csv('{}/{}'.format(files_along_x, file),
			header=None, skiprows=9)
		colname = ['x', 'y', 'z', 'ElField']  # mm, mm, mm, kV/mm
		this_df.columns = colname
		fname = re.findall(r'(.+).csv', file)[0]
		run = re.findall(r'\.h(\d*\.\d)\.', file)[0]
		hmax = run
		# hmax = re.findall(r'hmax(\d*)', file)[0]
		this_df['ID'] = fname
		this_df['run'] = run
		this_df['hmax'] = int(hmax[-1])
		# print(df.head())
		# print(this_df.head())
		this_df = this_df.sort_values(by=['x'])
		this_df = this_df.reset_index()
		df = df.append(this_df, sort=True)


df = df.sort_values(by=['ID'])
print(df.head())
# replace NAN with 0
df['hmax'] = df['run']
print(df[df.isnull().any(axis=1)])
# asser that no nonzero entries in the df
assert len(df[df.isnull().any(axis=1)]) == 0



Processing file: 01.MR.h5.0.lineData.csv
Processing file: 01.MR.h2.0.lineData.csv
Processing file: 01.MR.h1.0.lineData.csv
Processing file: 01.MR.h0.8.lineData.csv
Processing file: 01.MR.h20.0.lineData.csv
Processing file: 01.MR.h0.9.lineData.csv
          ElField                   ID  hmax  index  run          x  y  z
31   0.000000e+00  01.MR.h0.8.lineData     8     11  0.8  -0.171099  0  0
236  2.221414e+06  01.MR.h0.8.lineData     8    269  0.8  60.496321  0  0
235  2.111228e+06  01.MR.h0.8.lineData     8    265  0.8  58.865596  0  0
234  2.053113e+06  01.MR.h0.8.lineData     8    264  0.8  57.926956  0  0
233  2.040581e+06  01.MR.h0.8.lineData     8    266  0.8  57.718724  0  0
     ElField                    ID  hmax  index   run          x  y  z
323      NaN   01.MR.h0.8.lineData   0.8    324   0.8  78.565459  0  0
322      NaN   01.MR.h0.8.lineData   0.8    327   0.8  78.269065  0  0
321      NaN   01.MR.h0.8.lineData   0.8    323   0.8  77.458089  0  0
320      NaN   01.MR.h0.8

AssertionError: 

In [34]:
# check if the min and max of the x are the same
def minMaxX(df):
	my_ID = df.ID.unique()[0]
	print('Processing ID: {}'.format(my_ID))
	print(np.max(df['x']), np.min(df['x']))


# df.groupby('ID').apply(lambda x: minMaxX(x))


# group by ID and compute the differences in the profile
# reference dataframe with the finest mesh

# comment one of them !
# ref_df = df[ df['ID'] == '101_mr_0.5' ]   # new target
ref_df = df[df['hmax'] == str(hmax_reference)]
# print(ref_df)
E_ref_interp = interp1d(ref_df['x'], ref_df['ElField'],
	fill_value='extrapolate')


# PLOT ACCELERATOR COLUMN

my_plots = []

X = np.linspace(0, X_max, 400)  # query points
E_ref = E_ref_interp(X)

f, ax = plt.subplots(figsize=(6, 8))
f.suptitle('Mesh refinement in accelerator column')


def plotDifferences(df):
	my_ID = df.ID.unique()[0]
	hmax = df.hmax.unique()[0]
	if my_ID != ref_df.ID.unique()[0]:

		# el field interpolation
		E_interp = interp1d(df['x'], df['ElField'], fill_value='extrapolate')

		this_E = E_interp(X)

		diff_E = np.abs(np.abs(this_E - E_ref) / E_ref)

		# get label
		lbl = hmax

		this_plot1, = ax.plot(X, 100 * diff_E, label=lbl)
		ax.set_xlabel('x position [mm]')
		ax.set_ylim(0, ylimit)
		ax.set_ylabel('rel. diff. electric field [%]')
		ax.grid(True)

		my_plots.append(this_plot1)


# print(df)
df.groupby('ID').apply(lambda x: plotDifferences(x))

# legend
handles, labels = ax.get_legend_handles_labels()
# sort both labels and handles by labels
# print(labels)
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles[0:], labels[0:], title='hmax')

filename = '{}/mr_accelerator_column'.format(files_along_x)
# plt.savefig(filename + '.eps', dpi=1200)
# plt.savefig(filename + '.svg', dpi=1200)
plt.savefig(filename + '.png', dpi=600)
plt.close('all')


# PLOT ION EXTRACTION REGION

X = np.linspace(0, 11, 200)  # query points

E_ref = E_ref_interp(X)

f, ax = plt.subplots(figsize=(6, 8))
f.suptitle('Mesh refinement in extraction region column')


def plotDifferences(df):
	my_ID = df.ID.unique()[0]
	hmax = df.hmax.unique()[0]
	if my_ID != ref_df.ID.unique()[0]:
		# el field interpolation
		E_interp = interp1d(df['x'], df['ElField'], fill_value='extrapolate')

		this_E = E_interp(X)

		diff_E = np.abs(np.abs(this_E - E_ref) / E_ref)

		# get label
		lbl = hmax

		ax.plot(X, 100 * diff_E, label=lbl)
		ax.set_xlabel('x position [mm]')
		ax.set_ylim(0, ylimit)
		ax.set_ylabel('rel. diff. electric field [%]')
		ax.grid(True)


df.groupby('ID').apply(lambda x: plotDifferences(x))


# legend
handles, labels = ax.get_legend_handles_labels()
# sort both labels and handles by labels
labels, handles = zip(*sorted(zip(labels, handles), key=lambda t: t[0]))
ax.legend(handles[0:], labels[0:], title='hmax')

# plt.show()
filename = '{}/mr_extraction_region'.format(files_along_x)
# plt.savefig(filename + '.eps', dpi=1200)
# plt.savefig(filename + '.svg', dpi=1200)
plt.savefig(filename + '.png', dpi=600)
plt.close('all')

/Users/hkromer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in true_divide
/Users/hkromer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in true_divide
